In [1]:
FILE_PATH = '/home/matt/repos/ohia.ai/data'

In [2]:
model_name = 'mobilenetv2'
dataset = 'plantnet'

seed = 1
batch_size = 32

filtered = False
augmentation = False

n_thread = 32
gpu = 0

save_model = False

In [6]:
# set up paths
IMAGE_DIR = 'filtered_plantnet'
model_path = (
    f'{FILE_PATH}/models/' + 
    f'{dataset}' +
    ('_filtered' if filtered else '') +
    f'_seed-{seed}' +
    f'_batch_size-{batch_size}' +
    ('_augmentation' if augmentation else '')
)
print(model_path)
make_dir(model_path)

/home/matt/repos/ohia.ai/data/models/plantnet_seed-1_batch_size-32


In [5]:
# get list of images and labels
file_list = glob.glob(f'{FILE_PATH}/{IMAGE_DIR}/**/*.jpg', recursive=True)
full_label_list = [re.split('/', f)[-2] for f in file_list]

# encode label names with ids
fle = FastLabelEncoder()
label_ids = fle.fit_transform(full_label_list)

# save id2label map
id2label = {int(fle.transform([label])):label for label in np.unique(full_label_list)}
with open(f'{model_path}/plantnet_classes.json', 'w') as fp:
    json.dump(id2label, fp)

In [6]:
# split data
train_files, valid_files, train_ids, valid_ids \
    = train_test_split(file_list, label_ids, test_size=0.1, random_state=seed)

# create generators
n_classes = len(np.unique(full_label_list))
train_generator = PlantNetGenerator(
    train_files, train_ids, n_classes,
    batch_size=batch_size,
    augment=augmentation
)
valid_generator = PlantNetGenerator(
    valid_files, valid_ids, n_classes,
    batch_size=batch_size,
    augment=augmentation,
    shuffle=False
)

In [7]:
# define callbacks
callbacks_list = [
    callbacks.EarlyStopping(
        monitor='val_top_3_accuracy',
        patience=10,
        verbose=1,
        mode='max',
    ),
    callbacks.ReduceLROnPlateau(
        monitor='val_top_3_accuracy',
        factor=0.25,
        patience=2,
        verbose=1,
        mode='max',
    ),
    callbacks.ModelCheckpoint(
        monitor='val_top_3_accuracy',
        filepath=f'{model_path}/weights' + '_{epoch:02d}.h5',
        save_best_only=True,
        save_weights_only=False,
        mode='max',
    ),
]

In [8]:
def get_model(model_name, n_classes, n_hidden, dropout):

    from keras import layers, Model
    from keras.optimizers import Adam
    from keras.applications.mobilenet import MobileNet
    from keras.applications.mobilenetv2 import MobileNetV2
    from keras.applications.nasnet import NASNetMobile
    from ohia.metrics import top_1_accuracy, top_3_accuracy, top_5_accuracy

    import keras.backend as K
    K.clear_session()
    
    # load pretrained ImageNet model
    if model_name == 'mobilenetv1':
        base_model = MobileNet(
            input_shape=(224,224,3),
            weights='imagenet',
            include_top=False
        )        
    elif model_name == 'mobilenetv2':
        base_model = MobileNetV2(
            input_shape=(224,224,3),
            alpha=1.4,
            weights='imagenet',
            include_top=False
        )
    elif model_name == 'nasnetmobile':
        base_model = NASNetMobile(
            input_shape=(224,224,3),
            weights='imagenet',
            include_top=False
        )
    else:
        assert ValueError(
            f'model_name parameter must be one of the following'
            ' "mobilenetv1",'
            ' "mobilenetv2",'
            ' "nasnetmobile"'
        )

#     # set freeze all layers
#     for layer in base_model.layers:
#         if re.findall('_\\d_', layer.name):
#             print(layer.name)
#             layer.trainable = False
        
    # map ImageNet features to plants
    x = base_model.output
    x = layers.GlobalAveragePooling2D()(x)
    if dropout>0:
        x = layers.Dropout(dropout)(x)
    outputs = layers.Dense(n_classes, activation='softmax')(x)

    # compile the model
    model = Model(inputs=base_model.input, outputs=outputs)
    model.compile(
        optimizer=Adam(lr=0.0001),
        loss='categorical_crossentropy',
        metrics=[top_1_accuracy, top_3_accuracy, top_5_accuracy]
    )

    return model

In [11]:
# train model 
model = get_model(model_name, n_classes, n_hidden, dropout)
model.fit_generator(
    generator=train_generator,
    validation_data=valid_generator,
    callbacks=callbacks_list,
    use_multiprocessing=True,
    workers=n_thread,
    epochs=100,    
)

Epoch 1/100
2463/2463 [==============================] - 765s 311ms/step - loss: 3.1249 - top_1_accuracy: 0.3654 - top_3_accuracy: 0.5146 - top_5_accuracy: 0.5782 - val_loss: 2.3217 - val_top_1_accuracy: 0.4839 - val_top_3_accuracy: 0.6438 - val_top_5_accuracy: 0.7128
Epoch 2/100
2463/2463 [==============================] - 760s 309ms/step - loss: 1.5972 - top_1_accuracy: 0.6312 - top_3_accuracy: 0.7777 - top_5_accuracy: 0.8274 - val_loss: 1.9982 - val_top_1_accuracy: 0.5472 - val_top_3_accuracy: 0.7106 - val_top_5_accuracy: 0.7687
Epoch 3/100
2463/2463 [==============================] - 772s 313ms/step - loss: 1.0751 - top_1_accuracy: 0.7423 - top_3_accuracy: 0.8643 - top_5_accuracy: 0.9011 - val_loss: 1.9070 - val_top_1_accuracy: 0.5702 - val_top_3_accuracy: 0.7226 - val_top_5_accuracy: 0.7774
Epoch 4/100
2463/2463 [==============================] - 761s 309ms/step - loss: 0.7501 - top_1_accuracy: 0.8150 - top_3_accuracy: 0.9166 - top_5_accuracy: 0.9435 - val_loss: 1.7947 - val_top_1

FileNotFoundError: [Errno 2] No such file or directory: '/home/matt/repos/ohia.ai/data/filtered_plantnet/12807/337211.jpg'

In [ ]:
# save results
results = pd.DataFrame(model.history.history)
results.to_csv(f'{model_path}/results.csv', index=False)

In [ ]:
# print best results
print(results.iloc[results.val_top_3_accuracy.values.argmax()])